# Dataset Cleaning

### Games Meta Data

In [ ]:
# Source datasets required to rerun notebooks are not included in the repo due to size constraints.
# Dataset URL: https://amazon-reviews-2023.github.io/ << Video_Games

# Note: Data cleaning notebooks must be run sequentially (1, 2, 3).

In [ ]:
# Cleaning 137.269 games data aggressively to reduce size
# Strict handling of missing data to retain high-quality entries

In [ ]:
# Imports

import pandas as pd

In [ ]:
# Reset to show all columns + rows

pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')

### Data Exploration

In [ ]:
games_data = pd.read_json('../data/video_games_data.jsonl', lines = True)

games_data.shape

In [ ]:
games_data.sample()

In [ ]:
games_data.columns

In [ ]:
games_data.head(1)

In [ ]:
games_data.main_category.value_counts()

### Data Cleaning

In [ ]:
# Keep only 'Video Games'

games_data = games_data[games_data['main_category'] == 'Video Games']

# Drop 'main_category' column

games_data.drop('main_category', axis = 1, inplace = True)

games_data.shape

In [ ]:
games_data.sample()

In [ ]:
# Verify that all games have unique entries

len(games_data['parent_asin'].unique())

In [ ]:
games_data['author'].isna().sum()

In [ ]:
# Drop column 'author' because all its values are NaN

games_data.drop('author', axis = 1, inplace = True)

games_data.shape

In [ ]:
games_data.subtitle.isna().sum()

In [ ]:
# Drop column 'subtitle' because all its values are NaN

games_data.drop('subtitle', axis = 1, inplace = True)

games_data.shape

In [ ]:
games_data.bought_together.isna().sum()

In [ ]:
# Drop column 'bought_together' because all its values are NaN

games_data.drop('bought_together', axis = 1, inplace = True)

games_data.shape

In [ ]:
games_data.videos.value_counts()

In [ ]:
# Count rows without 'videos'

empty_videos_count = games_data['videos'].apply(lambda x: x == []).sum()

print('No video provided:', empty_videos_count)

In [ ]:
# Drop 'videos' due to insufficient data

games_data.drop('videos', axis = 1, inplace = True)

games_data.shape

In [ ]:
games_data.sample()

In [ ]:
games_data.rating_number.value_counts()

In [ ]:
# Rating Thresholding

games_data = games_data[games_data['rating_number'] >= 100]

games_data.shape

In [ ]:
games_data.description.value_counts()

In [ ]:
# Count rows without 'description'

empty_description = games_data['description'].apply(lambda x: x == []).sum()

print('No description provided:', empty_description)

In [ ]:
# Removing games without description

games_data = games_data[games_data['description'].apply(lambda x: x != [])]

games_data.shape

In [ ]:
games_data.columns

In [ ]:
games_data.features.value_counts()

In [ ]:
# Count rows without 'features'

empty_features = games_data['features'].apply(lambda x: x == []).sum()

print('No features provided:', empty_features)

In [ ]:
# Removing games without features

games_data = games_data[games_data['features'].apply(lambda x: x != [])]

games_data.shape

In [ ]:
games_data.sample()

In [ ]:
games_data.sample()['images']

In [ ]:
# Function to only keep first image URL

def extract_first_url(images_string):
    try:
        # Access the first dictionary
        images_list = images_string[0]
        
        # Return the 'large' URL if available
        return images_list['large']
        
    except (ValueError, IndexError):
        return None  # Handle malformed data gracefully

games_data['images'] = games_data['images'].apply(extract_first_url)

games_data['images'].value_counts()

In [ ]:
games_data.isna().sum()

In [ ]:
games_data.shape

In [ ]:
# Remove games without store
# Remove games without price

games_data = games_data[~games_data['store'].isna()]
games_data = games_data[~games_data['price'].isna()]

games_data.shape

In [ ]:
games_data.sample()

In [ ]:
# Remove 'average_rating' and 'rating_number'
# This information is found in the reviews table

games_data.drop(columns = ['average_rating', 'rating_number'], inplace = True)

games_data.shape

In [ ]:
# Show sample details entries

games_data.sample()['details'].iloc[0]

In [ ]:
# Get 'details' -> 'Type of item' distribution

item_types_count = {}

def types_count(row):
    value = row.get('Type of item')
    if value in item_types_count.keys():
        item_types_count[value] += 1
    else:
        item_types_count[value] = 0

games_data['details'].apply(types_count)

item_types_count

In [ ]:
# Only keep data of type 'Video Game'

games_data = games_data[games_data['details'].apply(lambda x: x.get('Type of item') == 'Video Game')]

games_data.shape

In [ ]:
# Get 'details' column key distribution

details_entries = {}

def check_keys(row):
    for key in row.keys():
        if key in details_entries.keys():
            details_entries[key] += 1
        else:
            details_entries[key] = 0

games_data['details'].apply(check_keys)

details_entries

In [ ]:
# Only keep ...
# 'Release date', 'Type of item', 'Is Discontinued By Manufacturer', 
# 'Manufacturer', 'Date First Available', 'Language'

keep_details = ['Release date', 'Type of item', 'Is Discontinued By Manufacturer', 
                'Manufacturer', 'Date First Available', 'Language']

def remove_detail_keys(row):
    # Safely remove keys if they exist
    for key in list(row.keys()):
        if key not in keep_details:
            del row[key]
    return row

games_data['details'] = games_data['details'].apply(remove_detail_keys)

games_data.sample()

In [ ]:
# Show sample 'categories' entries

games_data.sample()['categories'].iloc[0]

In [ ]:
# Get 'categories' column key distribution

category_entries = {}

def get_categories(row):
    for key in row:
        if key in category_entries:
            category_entries[key] += 1
        else:
            category_entries[key] = 0

games_data['categories'].apply(get_categories)

category_entries

In [ ]:
# Keep games in main categories only

main_cats = ['Video Games', 'PlayStation 4', 'Games', 'PC', 'PlayStation 3', 'Xbox One', 'Nintendo Switch', 
             'PlayStation', 'PlayStation 2', 'Nintendo DS', 'GameCube', 'Wii', 'Xbox 360', 'Wii U', 
             'PlayStation 5', 'Nintendo 64', 'Sony PSP', 'Xbox Series X & S', 'Nintendo NES', 'Xbox']

def keep_main_categories(row):
    for key in row:
        if key not in main_cats:
            return False
    return True

print(games_data.shape)

games_data = games_data[games_data['categories'].apply(keep_main_categories)]

games_data.shape

In [ ]:
# Remove hardware, toys, and accessories

games_data.reset_index(drop = True, inplace = True)

hw_toys_acc = [22, 29, 30, 54, 101, 109, 131, 221, 261, 305, 309, 312, 345, 347, 388, 399, 425, 452, 459,
                496, 527, 535, 545, 558, 567, 579, 590, 607, 614, 652, 658, 661, 666, 673, 684, 692, 695, 
                718, 731, 752, 761, 770, 772, 784, 824, 828, 898, 904, 936, 941, 946, 963, 970, 1041, 1042, 
                1045, 1058, 1066, 1085, 1110, 1114, 1142, 1143, 1174, 1180, 1182, 1184, 1197, 1209, 1220, 
                1229, 1233, 1265, 1270, 1284, 1286, 1374, 1375, 1376, 1410, 1457, 1458, 1482, 1487, 1502, 
                1504, 1507, 1509, 1516, 1568, 1575, 1632, 1645, 1647, 1662, 1689, 1690, 1691, 1692, 1711, 
                1717, 1727, 1772, 1778, 1782, 1792, 1799, 1824, 1900, 1959, 1962, 1864, 1970, 2012, 2017, 
                2027, 2030, 2050, 2074, 2075, 2097, 2100, 2125, 2132, 2133, 2151, 2194, 2218, 2235, 2240, 
                2255, 2270, 2275, 2293, 2301, 2302, 2309, 2325, 2327, 2333, 2354, 2357, 2374, 2381, 2401, 
                2415, 2416, 2445, 2446, 2460, 2466, 2472, 2547, 2530, 2587, 2594, 2599, 2600, 2621, 2627, 
                2628, 2643, 2649, 2667, 2670, 2686, 2704, 2710, 2713, 2717, 2719, 2720, 2724, 2739, 2745, 
                2771, 2793, 2798, 2801, 2808, 2812, 2816, 2834, 2845, 2861, 2896, 2903, 2936, 2943]

games_data = games_data.drop(index = hw_toys_acc)

games_data.shape

### Save Games Meta Data

In [ ]:
# Reset index

games_data.reset_index(drop = True)

# Save cleaned games meta data

games_data.to_csv('../data/meta_cleaned.csv', index = False)